In [84]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import mapping
import folium
import gmaps
import os
import requests
import shutil

In [85]:
with open('../resources/key.txt') as f:
    token = f.read()
    
gmaps.configure(api_key = token)

In [86]:
input_folder = '../data/railways/mbta_rapid_transit/'
input_file = input_folder+'MBTA_ARC.shp'

output_folder = '../data/output_images/BostonMTBA/'
output_csv = output_folder+'BOS.csv'

update_folder = '../data/output_images/BostonMTBA/set_3/'

### Import Routes

In [87]:
routes = gpd.read_file(input_file)

routes = routes.to_crs({'init' :'epsg:4269'})

routes.head()

,LINE,ROUTE,GRADE,SHAPE_LEN,geometry
0,GREEN,C D,7,458.386096,LINESTRING (-71.10139485815532 42.347577953996...
1,GREEN,B C D E,7,2121.006147,LINESTRING (-71.07920620455907 42.349579702367...
2,RED,A - Ashmont B - Braintree C - Alewife,7,7955.708839,LINESTRING (-71.08197671857094 42.361992845118...
3,GREEN,C E,7,1241.873692,LINESTRING (-71.05945866821224 42.359296597842...
4,GREEN,B C D,7,1466.201312,LINESTRING (-71.09616476184985 42.348960933708...


In [88]:
routes.shape

(136, 5)

In [90]:
'''
Get all points
'''

points = []
rows = routes.index.tolist()

geom_types = routes['geometry'].tolist()
for i in rows:
    try:
        route = routes.loc[i]
        if route.geometry.geom_type == 'LineString':
            g = route.geometry
            sec_points = mapping(g)["coordinates"]
            for j in sec_points:
                tmp = (j[0],j[1])
                points.append(tmp)
        elif route.geometry.geom_type == 'MultiLineString':
            g = route.geometry
            tmp = mapping(g)["coordinates"]
            for sec_points in tmp:
                for j in sec_points:
                    tmp = (j[0],j[1])
                    points.append(tmp)
    except Exception as e:
        print("Skipped route number",i,'because',e)        
        
    
print(len(points),'points')   

3441 points


In [89]:
'''
Open csv with current points
'''

curr_df = pd.read_csv(output_csv)
curr_df.head()

,Name,Longitude,Latitude,Catenary
0,-70.99141024363398_42.39987983654313,-70.991410,42.399880,1
1,-70.9916645142435_42.41343348702443,-70.991665,42.413433,1
2,-70.99169316827988_42.3987030963677,-70.991693,42.398703,1
3,-70.99195436847552_42.40432141901854,-70.991954,42.404321,1
4,-70.99217754599157_42.39778478905404,-70.992178,42.397785,1


In [95]:
'''
Get subselection of points excuding current points
'''
curr_points = list(zip(curr_df['Longitude'].tolist(),curr_df['Latitude'].tolist()))
# curr_points

route_points = []
for point in range(0,len(points),34):
    tmp = points[point]
    if tmp not in curr_points:
        route_points.append(tmp)

print(len(route_points))

102


In [96]:
'''
Create dataframe
'''

df = pd.DataFrame(route_points, columns=['Longitude','Latitude'])

tmp = list(df.Latitude.tolist())
names = []
for i in range(0,len(tmp)):
    name = str(df.iloc[i].Longitude)+'_'+str(df.iloc[i].Latitude)
    names.append(name)

df['Name'] = names

columns = ['Name','Longitude','Latitude']
df= df[columns]

df.head()

,Name,Longitude,Latitude
0,-71.10139485815532_42.34757795399692,-71.101395,42.347578
1,-71.06468391964727_42.35278078367277,-71.064684,42.352781
2,-71.05945866821224_42.359296597842494,-71.059459,42.359297
3,-71.11345563893369_42.37036645607184,-71.113456,42.370366
4,-71.12018649699813_42.3778930163705,-71.120186,42.377893


In [98]:
'''
Export points
'''

# Update
with open(output_csv, 'a') as f:
    df.to_csv(f, header=False,index=None)

# Write new
# df.to_csv(output_csv,header=True,index=None)
# df.to_csv('tmp.csv',header=True,index=None)

In [97]:
'''
Plot markers
'''

rows = df.index.tolist()

marker_points = []
for row in rows:
    marker_points.append((df.iloc[row].Latitude,df.iloc[row].Longitude))
    
# Set map centerpoint
coords = marker_points[0]
       
# Define map
m = folium.Map(
    location = coords,
    zoom_start = 13
)
  
# Add points    
for mp in marker_points:
    folium.Marker(mp).add_to(m)
m

In [108]:
'''
Get satellite preview for image
'''

row = 0

figure_layout = {
    'width': '100%',
    'height': '800px'
}
fig = gmaps.figure(center = (df.iloc[row].Latitude,df.iloc[row].Longitude), zoom_level = 19,map_type='SATELLITE',layout=figure_layout)
fig

Figure(layout=FigureLayout(height='800px', width='100%'))

In [99]:
'''
Get all images 
'''

root = os.path.dirname(os.path.abspath('Africa.ipynb'))
image_folder = root[:-9]+update_folder[3:]
image_folder

url = 'https://maps.googleapis.com/maps/api/staticmap?'
rows = df.index.tolist()
for i in range(0,len(rows)):
    row = df.iloc[i]
    center = str(row.Latitude)+','+str(row.Longitude)
    payload = {
        'center': center, 
        'zoom': 20,
        'size': '640x640',
        'scale': 2,
        'format': 'png32',
        'maptype': 'satellite',
        'key': token
    }
    
    r = requests.get(url,params=payload,stream=True)

    name = df.iloc[i]['Name']
    filename = image_folder+name+'.png'
    if r.status_code == 200:
        with open(filename, 'wb') as f:
            r.raw.decode_content = True
            shutil.copyfileobj(r.raw, f) 